In [29]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [30]:
## Load model, scaler, and encoders

ANN_model = load_model('model.keras')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('geography_OHE.pkl', 'rb') as file:
    geography_OHE = pickle.load(file)

with open('gender_LE.pkl', 'rb') as file:
    gender_LE = pickle.load(file)

c:\softwareDevelopment\Projects\Churning_ANN\venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [31]:
# Sample Data
input_data = {
    'CreditScore': 750,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 3,
    'Balance': 40000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 65000
}

In [ ]:
# Convert input data to dataframe

input_data = pd.DataFrame([input_data])

In [33]:
## Encode Geography Input Column

geo_encoded = geography_OHE.transform(input_data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geography_OHE.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
## Encode Gender Input Column

gender_encoded = gender_LE.transform(input_data['Gender'])
input_data['Gender'] = gender_encoded
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,France,1,35,3,40000,2,1,1,65000


In [ ]:
## Replace Geography column with encoded values

input_data = pd.concat([input_data.drop(['Geography'], axis=1), geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,1,35,3,40000,2,1,1,65000,1.0,0.0,0.0


In [47]:
## Scale the data

input_data_scaled = scaler.transform(input_data)

In [48]:
prediction = ANN_model.predict(input_data_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [ ]:
print(prediction[0][0])s

0.016047178


In [51]:
if prediction[0][0] < 0.5:
    print('The customer is not likely to churn')
else:
    print('The customer is likely to churn')

The customer is not likely to churn
